# **VIDEO SUMMERIZATION: GENERATING MOVIE TRAILER (MINI PORJECT) **

In [1]:
# 📦 Install required libraries
!pip install opencv-python numpy ffmpeg-python librosa

In [10]:

import cv2
import numpy as np
import os

def extract_keyframes(video_path, output_folder, min_frame_gap=100):
    cap = cv2.VideoCapture(video_path)
    prev_frame = None
    frame_count = 0
    last_saved = -min_frame_gap  # so first frame passes
    saved_frames = []

    os.makedirs(output_folder, exist_ok=True)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_frame is not None:
            diff = cv2.absdiff(prev_frame, gray)
            mean_diff = np.mean(diff)

            # Save only if frame is different and far from last one saved
            if mean_diff > 25 and (frame_count - last_saved) >= min_frame_gap:
                frame_filename = os.path.join(output_folder, f"frame_{frame_count}.jpg")
                cv2.imwrite(frame_filename, frame)
                saved_frames.append(frame_count)
                last_saved = frame_count

        prev_frame = gray
        frame_count += 1

    cap.release()
    print(f"Extracted {len(saved_frames)} spaced keyframes.")
    return saved_frames


In [11]:
# 🎞️ Step 2: Generate 2-Second Clips from Selected Frames
import subprocess

def generate_clips(video_path, frames, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()

    for frame in frames:
        timestamp = frame / fps
        clip_name = os.path.join(output_folder, f"frame_{frame}.mp4")
        command = f"ffmpeg -y -ss {timestamp} -t 2 -i {video_path} -vf scale=640:360 -preset veryfast -crf 28 -c:v libx264 -c:a aac {clip_name}"


        subprocess.run(command, shell=True)
    print("Clips generated.")

import os
import subprocess

def create_trailer_with_transitions(clips_folder, frames, output_trailer):
    # Step 1: Generate input list for filtering
    filter_complex = ""
    inputs = []
    for idx, frame in enumerate(frames):
        clip = os.path.join(clips_folder, f"frame_{frame}.mp4")
        inputs.append(f"-i {clip}")
        filter_complex += f"[{idx}:v][{idx}:a]"

    # Build crossfade filters
    fade_filters = ""
    for i in range(len(frames) - 1):
        fade_filters += (
            f"[v{i}][a{i}][{i+1}:v][{i+1}:a]xfade=transition=fade:duration=1:offset={2*i}:"
            f"format=yuv420p[v{i+1}][a{i+1}];"
        )

    # Final output mapping
    last_video = f"v{len(frames) - 1}"
    last_audio = f"a{len(frames) - 1}"

    # Complete FFmpeg command
    inputs_str = " ".join(inputs)
    cmd = f"""
    ffmpeg {inputs_str} -filter_complex "
    {' '.join([f'[{i}:v]scale=640:360,setpts=PTS-STARTPTS[v{i}];[{i}:a]asetpts=PTS-STARTPTS[a{i}]' for i in range(len(frames))])};
    {fade_filters[:-1]}
    [{last_video}][{last_audio}]concat=n=1:v=1:a=1[outv][outa]
    " -map "[outv]" -map "[outa]" {output_trailer}
    """
    subprocess.run(cmd, shell=True)
    print(f"Trailer with transitions created at: {output_trailer}")


In [12]:
# ▶️ Run the entire pipeline
video_file = "/content/VIDEO-2025-03-10-00-06-48.mp4"  # Replace with your movie file
keyframe_folder = "keyframes"
clips_folder = "clips"


import os
import subprocess

def create_trailer_with_transitions(clips_folder, frames, output_trailer):
    # Step 1: Generate input list for filtering
    filter_complex = ""
    inputs = []
    for idx, frame in enumerate(frames):
        clip = os.path.join(clips_folder, f"frame_{frame}.mp4")
        inputs.append(f"-i {clip}")
        filter_complex += f"[{idx}:v][{idx}:a]"

    # Build crossfade filters
    fade_filters = ""
    for i in range(len(frames) - 1):
        fade_filters += (
            f"[v{i}][a{i}][{i+1}:v][{i+1}:a]xfade=transition=fade:duration=1:offset={2*i}:"
            f"format=yuv420p[v{i+1}][a{i+1}];"
        )

    # Final output mapping
    last_video = f"v{len(frames) - 1}"
    last_audio = f"a{len(frames) - 1}"

    # Complete FFmpeg command
    inputs_str = " ".join(inputs)
    cmd = f"""
    ffmpeg {inputs_str} -filter_complex "
    {' '.join([f'[{i}:v]scale=640:360,setpts=PTS-STARTPTS[v{i}];[{i}:a]asetpts=PTS-STARTPTS[a{i}]' for i in range(len(frames))])};
    {fade_filters[:-1]}
    [{last_video}][{last_audio}]concat=n=1:v=1:a=1[outv][outa]
    " -map "[outv]" -map "[outa]" {output_trailer}
    """
    subprocess.run(cmd, shell=True)
    print(f"Trailer with transitions created at: {output_trailer}")


print("Extracting keyframes...")
keyframes = extract_keyframes(video_file, keyframe_folder, min_frame_gap=100)


import random

selected_frames = random.sample(keyframes, min(40, len(keyframes)))  # 75 clips × 2s = 150s = 2.5 minutes

#selected_frames.sort()  # for smooth viewing



generate_clips(video_file, selected_frames, clips_folder)
create_trailer_with_transitions(clips_folder, selected_frames, "final_trailer.mp4")

Extracting keyframes...
Extracted 0 spaced keyframes.
Clips generated.
Trailer with transitions created at: final_trailer.mp4


In [13]:
import os
import subprocess

def check_clip_durations(clips_folder):
    for file in os.listdir(clips_folder):
        if file.endswith(".mp4"):
            path = os.path.join(clips_folder, file)
            cmd = f"ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1 \"{path}\""
            result = subprocess.getoutput(cmd)
            print(f"{file}: {result.strip()} sec")

check_clip_durations("clips")
